In [ ]:
%pip install ultralytics

In [1]:
import torch
import os
import time
from ultralytics import YOLO
import random

In [2]:
"""model_path = "/home/jupyter/datasphere/project/logs/yolo_gpu/weights/best.pt"
model = YOLO(model_path)  # load a custom model

# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
print(metrics.box.map,  # map50-95
metrics.box.map50,  # map50
metrics.box.map75,  # map75
metrics.box.maps)  # a list contains map50-95 of each category"""

'model_path = "/home/jupyter/datasphere/project/logs/yolo_gpu/weights/best.pt"\nmodel = YOLO(model_path)  # load a custom model\n\n# Validate the model\nmetrics = model.val()  # no arguments needed, dataset and settings remembered\nprint(metrics.box.map,  # map50-95\nmetrics.box.map50,  # map50\nmetrics.box.map75,  # map75\nmetrics.box.maps)  # a list contains map50-95 of each category'

In [7]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [8]:
model_path = "/home/jupyter/datasphere/project/logs/yolo_gpu/weights/best.pt"
img_path = "/home/jupyter/datasphere/project/data/test/images"
bs = 16

images = [f"{img_path}/{i}" for i in os.listdir(img_path)]
images = random.sample(images, bs*100)
model = YOLO(model_path)
#res_list = []

start = time.time()
for b in batch(images, bs):
    r = model(b)
    #print(r.probs)
end = (time.time()-start)/(bs*len(images))/60.
print(f"bs={bs} time = {end} sec/img")


0: 640x640 4 drones, 4.4ms
1: 640x640 1 drone, 4.4ms
2: 640x640 (no detections), 4.4ms
3: 640x640 1 heli, 4.4ms
4: 640x640 1 plane, 4.4ms
5: 640x640 1 plane, 4.4ms
6: 640x640 1 heli, 4.4ms
7: 640x640 (no detections), 4.4ms
8: 640x640 1 drone, 4.4ms
9: 640x640 (no detections), 4.4ms
10: 640x640 1 heli, 4.4ms
11: 640x640 (no detections), 4.4ms
12: 640x640 (no detections), 4.4ms
13: 640x640 1 uav, 4.4ms
14: 640x640 1 plane, 4.4ms
15: 640x640 1 bird, 4.4ms
Speed: 2.2ms preprocess, 4.4ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 heli, 4.2ms
1: 640x640 1 drone, 4.2ms
2: 640x640 1 drone, 4.2ms
3: 640x640 1 bird, 4.2ms
4: 640x640 1 drone, 4.2ms
5: 640x640 1 heli, 4.2ms
6: 640x640 1 plane, 4.2ms
7: 640x640 1 drone, 4.2ms
8: 640x640 1 plane, 4.2ms
9: 640x640 1 plane, 4.2ms
10: 640x640 1 plane, 4.2ms
11: 640x640 1 heli, 4.2ms
12: 640x640 (no detections), 4.2ms
13: 640x640 1 plane, 4.2ms
14: 640x640 1 heli, 4.2ms
15: 640x640 3 uavs, 4.2ms
Speed: 2.2ms preproce

In [9]:
model = YOLO(model_path)

onnx_path = "/home/jupyter/datasphere/project/logs/yolo_gpu/weights/best.onnx"
model.export(format="onnx",
             imgsz=640,
             batch=bs,
             #half=True,
             #dynamic=True,
             #simplify=True,
             #opset=?,
             )  

onnx_model = YOLO(onnx_path)

start = time.time()
for b in batch(images, bs):
    r = onnx_model.predict(b, device="0")
end = (time.time()-start)/(bs*len(images))/60.
print(f"bs={bs} time = {end} sec/img")

Ultralytics YOLOv8.2.30 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon Processor (Icelake))
Model summary (fused): 168 layers, 3006623 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/home/jupyter/datasphere/project/logs/yolo_gpu/weights/best.pt' with input shape (16, 3, 640, 640) BCHW and output shape(s) (16, 9, 8400) (5.9 MB)

ONNX: starting export with onnx 1.16.1 opset 17...
============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

ONNX: export success ✅ 1.9s, saved as '/home/jupyter/datasphere/project/logs/yolo_gpu/weights/best.onnx' (11.7 MB)

Export complete (5.9s)
Results saved to /home/jupyter/work/resources/logs/yolo_gpu/weights
Predict:         yolo predict task=detect model=/home/jupyter/datasphere/project/logs/yolo_gpu/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=/home/jupyter/dataspher

In [23]:
model = YOLO(model_path)

onnx_path = "/home/jupyter/datasphere/project/logs/yolo_gpu/weights/best.onnx"
model.export(format="onnx",
             imgsz=640,
             batch=bs,
             half=True,
             #simplify=True,
             #opset=?,
             )  

onnx_model = YOLO(onnx_path)

start = time.time()
for b in batch(images, bs):
    r = onnx_model.predict(b, device="0")
end = (time.time()-start)/(bs*len(images))/60.
print(f"bs={bs} time = {end} sec/img")

Ultralytics YOLOv8.2.30 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon Processor (Icelake))
WARNING ⚠️ half=True only compatible with GPU export, i.e. use device=0
Model summary (fused): 168 layers, 3006623 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/home/jupyter/datasphere/project/logs/yolo_gpu/weights/best.pt' with input shape (16, 3, 640, 640) BCHW and output shape(s) (16, 9, 8400) (5.9 MB)

ONNX: starting export with onnx 1.16.1 opset 17...
============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

ONNX: export success ✅ 2.0s, saved as '/home/jupyter/datasphere/project/logs/yolo_gpu/weights/best.onnx' (11.7 MB)

Export complete (6.2s)
Results saved to /home/jupyter/work/resources/logs/yolo_gpu/weights
Predict:         yolo predict task=detect model=/home/jupyter/datasphere/project/logs/yolo_gpu/weights/best.onnx imgsz=6

In [22]:
model = YOLO(model_path)

onnx_path = "/home/jupyter/datasphere/project/logs/yolo_gpu/weights/best.onnx"
model.export(format="onnx",
             imgsz=640,
             batch=bs,
             #half=True,
             #simplify=True,
             int8=True,
             #opset=?,
             )  

onnx_model = YOLO(onnx_path)

start = time.time()
for b in batch(images, bs):
    r = onnx_model.predict(b, device="cuda")
end = (time.time()-start)/(bs*len(images))/60.
print(f"bs={bs} time = {end} sec/img")

Ultralytics YOLOv8.2.30 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon Processor (Icelake))
WARNING ⚠️ INT8 export requires a missing 'data' arg for calibration. Using default 'data=coco8.yaml'.
Model summary (fused): 168 layers, 3006623 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/home/jupyter/datasphere/project/logs/yolo_gpu/weights/best.pt' with input shape (16, 3, 640, 640) BCHW and output shape(s) (16, 9, 8400) (5.9 MB)

ONNX: starting export with onnx 1.16.1 opset 17...
============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

ONNX: export success ✅ 2.0s, saved as '/home/jupyter/datasphere/project/logs/yolo_gpu/weights/best.onnx' (11.7 MB)

Export complete (5.9s)
Results saved to /home/jupyter/work/resources/logs/yolo_gpu/weights
Predict:         yolo predict task=detect model=/home/jupyter/datasphere/project/logs/yol

IndexError: too many indices for array: array is 0-dimensional, but 3 were indexed

In [21]:
%pip install onnxruntime-gpu

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [7]:
%pip install -U ultralytics sahi

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 511.6 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 1.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.6/780.6 kB 7.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.6/112.6 kB 14.5 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117032 sha256=0b0779305705dfcc91af09a88bf1d2c6cbe5b7143827fee79baa869024f4cd0d
  Stored in directory: /tmp/xdg_cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.2.28
    Uninstalling ultralytics-8.2.28:
      

In [10]:
from pathlib import Path

from IPython.display import Image
from sahi import AutoDetectionModel
from sahi.predict import get_prediction, get_sliced_prediction, predict
from sahi.utils.cv import read_image
from sahi.utils.file import download_from_url
from sahi.utils.yolov8 import download_yolov8s_model

detection_model = AutoDetectionModel.from_pretrained(
    model_type="yolov8",
    model_path=model_path,
    confidence_threshold=0.25,
    device="cuda:0",  # or 'cuda:0'
)

start = time.time()
for i in images:
    result = get_sliced_prediction(
        i,
        detection_model,
        slice_height=640,
        slice_width=640,
        overlap_height_ratio=0.2,
        overlap_width_ratio=0.2,
        #model_confidence_threshold=0.25,
    )
    result.export_visuals(export_dir="/home/jupyter/datasphere/project/results_sahi/")
end = (time.time()-start)/(bs*len(images))/60.
print(f"bs={bs} time = {end} sec/img")

Performing prediction on 8 slices.
Performing prediction on 6 slices.
Performing prediction on 1 slices.
Performing prediction on 8 slices.
Performing prediction on 1 slices.
Performing prediction on 8 slices.
Performing prediction on 8 slices.
Performing prediction on 6 slices.
Performing prediction on 1 slices.
Performing prediction on 1 slices.
Performing prediction on 1 slices.
Performing prediction on 4 slices.
Performing prediction on 8 slices.
Performing prediction on 1 slices.
Performing prediction on 2 slices.
Performing prediction on 8 slices.
Performing prediction on 2 slices.
Performing prediction on 2 slices.
Performing prediction on 8 slices.
Performing prediction on 8 slices.
Performing prediction on 6 slices.
Performing prediction on 8 slices.
Performing prediction on 8 slices.
Performing prediction on 16 slices.
Performing prediction on 6 slices.
Performing prediction on 8 slices.
Performing prediction on 8 slices.
Performing prediction on 1 slices.
Performing predicti

KeyboardInterrupt: 

ModuleNotFoundError: No module named 'sahi.models.yolox_onnx'